In [101]:
#https://medium.com/@constandinou.antonio/quant-post-3-1-a-guided-path-into-mean-reversion-8b33b3c279e4
#https://medium.com/@bart.chr/pairs-trading-for-algorithmic-trading-breakdown-d8b709f59372

import pandas as pd
import statsmodels.api as sm
import numpy as np
import statsmodels.tsa.stattools as ts

In [33]:
data = pd.read_csv('datasets/data.csv')[['AALR3', 'ABCB4']]
data.columns = ['y', 'x']

In [90]:
x = sm.add_constant(data['x'])

# Fit and summarize OLS model
mod = sm.OLS(data['y'], x)
results = mod.fit()
print(results.params)
#print(results.summary())

const   -6.936373
x        1.308303
dtype: float64


In [51]:
halflife = round(-np.log(2))/results.params[1]
halflife

-0.7643490674975995

In [52]:
print('half life',-np.log(2)/results.params[1])

half life -0.5298064010995843


In [57]:
print(results.f_test(np.identity(2)))

<F test: F=array([[9701.85022323]]), p=7.864479046236036e-106, df_denom=90, df_num=2>


In [109]:
#https://groups.google.com/forum/#!topic/pystatsmodels/HbUIeSuXqZs
class DickeyFullerTest:
    def __init__(self):
        self.df = None
        self.crit = None
        self.slope = None
        self.halflife = None
        self.lookback = None
        self.coint = None
        
    def adf(self, y, x):

        model = sm.GLS(y,sm.add_constant(x)).fit()
        coint = model.resid

        print(model.params)

        adfstat, pvalue, critvalues, res = ts.adfuller(coint, store=True, regresults=True)

        self.df = model.params[1]/model.bse[1]
        self.crit = res.critvalues
        self.slope = model.params[1]
        self.halflife = -np.log(2)/model.params[1]
        self.lookback = int(round(-np.log(2)/model.params[1]))
        self.coint = coint
        
        print('df: ', self.df)
        #print('crit: ', self.crit)
        print('slope: ', self.slope)
        print('halflife: ', self.halflife)
        print('lookback: ', self.lookback)

In [87]:
x= []
x.append(['2007-07-24',1.03660])
x.append(['2007-07-25',1.04180])
x.append(['2007-07-26',1.05360])
x.append(['2007-07-27',1.06390])
x.append(['2007-07-30',1.06556])
x.append(['2007-07-31',1.06680])
x.append(['2007-08-01',1.05600])
x.append(['2007-08-02',1.05300])
x.append(['2007-08-03',1.05210])
x.append(['2007-08-06',1.05170])
x.append(['2007-08-07',1.05510])
x.append(['2007-08-08',1.04850])
x.append(['2007-08-09',1.05705])
x.append(['2007-08-10',1.05310])
dt_x = pd.DataFrame(x)
dt_x.columns=['data', 'x']

y = []
y.append(['2007-07-24',-0.01016])
y.append(['2007-07-25',0.00520])
y.append(['2007-07-26',0.01180])
y.append(['2007-07-27',0.01030])
y.append(['2007-07-30',0.00166])
y.append(['2007-07-31',0.00124])
y.append(['2007-08-01',-0.01080])
y.append(['2007-08-02',-0.00300])
y.append(['2007-08-03',-0.00090])
y.append(['2007-08-06',-0.00040])
y.append(['2007-08-07',0.00340])
y.append(['2007-08-08',-0.00660])
y.append(['2007-08-09',0.00855])
y.append(['2007-08-10',-0.00395])
dt_y = pd.DataFrame(y)
dt_y.columns=['data', 'y']

In [112]:
test = DickeyFullerTest()
test.adf(dt_y['y'], dt_x['x'])

const   -0.349714
x        0.332254
dtype: float64
df:  1.4773934468234595
slope:  0.3322535262683186
halflife:  -2.0861996209490314
lookback:  -2


In [113]:
test = DickeyFullerTest()
test.adf(data['y'], data['x'])

const   -6.936373
x        1.308303
dtype: float64
df:  45.48603661109386
slope:  1.3083027670510512
halflife:  -0.5298064010995843
lookback:  -1
